# Practical Application III: Comparing Classifiers

**Overview**: In this practical application, your goal is to compare the performance of the classifiers we encountered in this section, namely K Nearest Neighbor, Logistic Regression, Decision Trees, and Support Vector Machines.  We will utilize a dataset related to marketing bank products over the telephone.  



### Getting Started

Our dataset comes from the UCI Machine Learning repository [link](https://archive.ics.uci.edu/ml/datasets/bank+marketing).  The data is from a Portugese banking institution and is a collection of the results of multiple marketing campaigns.  We will make use of the article accompanying the dataset [here](CRISP-DM-BANK.pdf) for more information on the data and features.



### Problem 1: Understanding the Data

To gain a better understanding of the data, please read the information provided in the UCI link above, and examine the **Materials and Methods** section of the paper.  How many marketing campaigns does this data represent?

In [ ]:
# For p1 after reading the paper, I cannot figure out the exact number of marketing compaigns presented. However, I saw there are three iterations in the compaign loop
# the first one is to define the goal with its business understanding phase,
# the second one is to try optimize the number of classification labels by redefining the goal
# the third one is to minimize the dimensionality of parameters considered for training in order to fit more models and algos.

### Problem 2: Read in the Data

Use pandas to read in the dataset `bank-additional-full.csv` and assign to a meaningful variable name.

In [2]:
import pandas as pd

In [3]:
df = pd.read_csv('data/bank-additional-full.csv', sep = ';')

In [4]:
df.head()

,age,job,marital,education,default,housing,loan,contact,month,day_of_week,...,campaign,pdays,previous,poutcome,emp.var.rate,cons.price.idx,cons.conf.idx,euribor3m,nr.employed,y
0,56,housemaid,married,basic.4y,no,no,no,telephone,may,mon,...,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
1,57,services,married,high.school,unknown,no,no,telephone,may,mon,...,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
2,37,services,married,high.school,no,yes,no,telephone,may,mon,...,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
3,40,admin.,married,basic.6y,no,no,no,telephone,may,mon,...,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
4,56,services,married,high.school,no,no,yes,telephone,may,mon,...,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no


### Problem 3: Understanding the Features


Examine the data description below, and determine if any of the features are missing values or need to be coerced to a different data type.


```
Input variables:
# bank client data:
1 - age (numeric)
2 - job : type of job (categorical: 'admin.','blue-collar','entrepreneur','housemaid','management','retired','self-employed','services','student','technician','unemployed','unknown')
3 - marital : marital status (categorical: 'divorced','married','single','unknown'; note: 'divorced' means divorced or widowed)
4 - education (categorical: 'basic.4y','basic.6y','basic.9y','high.school','illiterate','professional.course','university.degree','unknown')
5 - default: has credit in default? (categorical: 'no','yes','unknown')
6 - housing: has housing loan? (categorical: 'no','yes','unknown')
7 - loan: has personal loan? (categorical: 'no','yes','unknown')
# related with the last contact of the current campaign:
8 - contact: contact communication type (categorical: 'cellular','telephone')
9 - month: last contact month of year (categorical: 'jan', 'feb', 'mar', ..., 'nov', 'dec')
10 - day_of_week: last contact day of the week (categorical: 'mon','tue','wed','thu','fri')
11 - duration: last contact duration, in seconds (numeric). Important note: this attribute highly affects the output target (e.g., if duration=0 then y='no'). Yet, the duration is not known before a call is performed. Also, after the end of the call y is obviously known. Thus, this input should only be included for benchmark purposes and should be discarded if the intention is to have a realistic predictive model.
# other attributes:
12 - campaign: number of contacts performed during this campaign and for this client (numeric, includes last contact)
13 - pdays: number of days that passed by after the client was last contacted from a previous campaign (numeric; 999 means client was not previously contacted)
14 - previous: number of contacts performed before this campaign and for this client (numeric)
15 - poutcome: outcome of the previous marketing campaign (categorical: 'failure','nonexistent','success')
# social and economic context attributes
16 - emp.var.rate: employment variation rate - quarterly indicator (numeric)
17 - cons.price.idx: consumer price index - monthly indicator (numeric)
18 - cons.conf.idx: consumer confidence index - monthly indicator (numeric)
19 - euribor3m: euribor 3 month rate - daily indicator (numeric)
20 - nr.employed: number of employees - quarterly indicator (numeric)

Output variable (desired target):
21 - y - has the client subscribed a term deposit? (binary: 'yes','no')
```



In [5]:
# we can replace all the unknowns with nan in order to void them
df.replace('unknown', pd.NA,inplace=True)

# for the numerica data, we can keep it as it is. For those categorical datas, it's actually stored as object, we can convert it to category type.
categorical_cols = [
    'job', 'marital', 'education', 'default', 'housing', 'loan',
    'contact', 'month', 'day_of_week', 'poutcome',
]
df[categorical_cols] = df[categorical_cols].astype('category')


### Problem 4: Understanding the Task

After examining the description and data, your goal now is to clearly state the *Business Objective* of the task.  State the objective below.

In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 41188 entries, 0 to 41187
Data columns (total 21 columns):
 #   Column          Non-Null Count  Dtype   
---  ------          --------------  -----   
 0   age             41188 non-null  int64   
 1   job             40858 non-null  category
 2   marital         41108 non-null  category
 3   education       39457 non-null  category
 4   default         32591 non-null  category
 5   housing         40198 non-null  category
 6   loan            40198 non-null  category
 7   contact         41188 non-null  category
 8   month           41188 non-null  category
 9   day_of_week     41188 non-null  category
 10  duration        41188 non-null  int64   
 11  campaign        41188 non-null  int64   
 12  pdays           41188 non-null  int64   
 13  previous        41188 non-null  int64   
 14  poutcome        41188 non-null  category
 15  emp.var.rate    41188 non-null  float64 
 16  cons.price.idx  41188 non-null  float64 
 17  cons.conf.id

In [7]:
# The goal will be to predict whether the client will subscribe the term deposit based on the datasets information given. We are aimed to train a model that helps us predict
# client's decision on the subscriptions

### Problem 5: Engineering Features

Now that you understand your business objective, we will build a basic model to get started.  Before we can do this, we must work to encode the data.  Using just the bank information features, prepare the features and target column for modeling with appropriate encoding and transformations.

In [11]:
import pandas as pd
from sklearn.preprocessing import LabelEncoder

# manually selected some possible related features for training
selected_features = ['job','marital','education','housing','loan','duration','campaign','pdays','previous','poutcome']
target = 'y'
df = df[selected_features + [target]]

# drop the missing values
df.dropna(inplace=True)

# One-hot encode categorical variables
df_encoded = pd.get_dummies(df.drop(columns=target), drop_first=True)

# Label encode the target variable
label_encoder = LabelEncoder()
df_encoded[target] = label_encoder.fit_transform(df[target])


X = df_encoded.drop(columns=target)
y = df_encoded[target]

print(X.head())
print(y.head())

# now we have the encoded datasets that transform catogorical data columns with one hot encoding, and the result mapping of 0 or 1 as no or yes. We can begin exploring the training
# steps.

   duration  campaign  pdays  previous  job_blue-collar  job_entrepreneur  \
0       261         1    999         0            False             False   
1       149         1    999         0            False             False   
2       226         1    999         0            False             False   
3       151         1    999         0            False             False   
4       307         1    999         0            False             False   

   job_housemaid  job_management  job_retired  job_self-employed  ...  \
0           True           False        False              False  ...   
1          False           False        False              False  ...   
2          False           False        False              False  ...   
3          False           False        False              False  ...   
4          False           False        False              False  ...   

   education_basic.6y  education_basic.9y  education_high.school  \
0               False         

### Problem 6: Train/Test Split

With your data prepared, split it into a train and test set.

In [12]:
from sklearn.model_selection import train_test_split


X_train, X_test, y_train, y_test = train_test_split(
    X, y,
    test_size=0.2,
    random_state=42,
    stratify=y
)

# ensure that the size matches for train and test X,y
print(f"Train set: {X_train.shape}, {y_train.shape}")
print(f"Test set:  {X_test.shape}, {y_test.shape}")


Train set: (30596, 26), (30596,)
Test set:  (7649, 26), (7649,)


### Problem 7: A Baseline Model

Before we build our first model, we want to establish a baseline.  What is the baseline performance that our classifier should aim to beat?

In [13]:
from sklearn.dummy import DummyClassifier
from sklearn.metrics import accuracy_score, classification_report

baseline = DummyClassifier(strategy="most_frequent", random_state=42)
baseline.fit(X_train, y_train)

y_pred_baseline = baseline.predict(X_test)

# performance evaluation
accuracy = accuracy_score(y_test, y_pred_baseline)
print(f"Baseline Accuracy: {accuracy:.4f}")
print("\nClassification Report:")
print(classification_report(y_test, y_pred_baseline))


Baseline Accuracy: 0.8886

Classification Report:
              precision    recall  f1-score   support

           0       0.89      1.00      0.94      6797
           1       0.00      0.00      0.00       852

    accuracy                           0.89      7649
   macro avg       0.44      0.50      0.47      7649
weighted avg       0.79      0.89      0.84      7649



C:\Users\jhw75\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
C:\Users\jhw75\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
C:\Users\jhw75\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\sklearn\metrics\_classificati

In [ ]:
# from the above code, the baseline model shows an accuracy of 0.8886, this would be the performance we aim to beat.

### Problem 8: A Simple Model

Use Logistic Regression to build a basic model on your data.  

In [14]:
from sklearn.linear_model import LogisticRegression
import time
log_reg = LogisticRegression(random_state=42, max_iter=1000) # Increased max_iter for convergence
start_time_log_reg = time.time()
log_reg.fit(X_train, y_train)
end_time_log_reg = time.time()
y_pred_log_reg = log_reg.predict(X_test)

time_trained_log_reg = end_time_log_reg - start_time_log_reg
print(f"Training Time: {time_trained_log_reg:.2f} seconds")

Training Time: 1.06 seconds


C:\Users\jhw75\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\sklearn\linear_model\_logistic.py:465: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. OF ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


### Problem 9: Score the Model

What is the accuracy of your model?

In [15]:
# model evaluation
train_accurarcy_log_reg = accuracy_score(y_train, log_reg.predict(X_train))
accuracy_log_reg = accuracy_score(y_test, y_pred_log_reg)
print(f"Training Accuracy: {train_accurarcy_log_reg:.4f}")
print(f"Logistic Regression Accuracy: {accuracy_log_reg:.4f}")
print("\nClassification Report:")
print(classification_report(y_test, y_pred_log_reg))

Training Accuracy: 0.9056
Logistic Regression Accuracy: 0.9068

Classification Report:
              precision    recall  f1-score   support

           0       0.92      0.98      0.95      6797
           1       0.67      0.33      0.44       852

    accuracy                           0.91      7649
   macro avg       0.79      0.65      0.69      7649
weighted avg       0.89      0.91      0.89      7649



### Problem 10: Model Comparisons

Now, we aim to compare the performance of the Logistic Regression model to our KNN algorithm, Decision Tree, and SVM models.  Using the default settings for each of the models, fit and score each.  Also, be sure to compare the fit time of each of the models.  Present your findings in a `DataFrame` similar to that below:

| Model | Train Time | Train Accuracy | Test Accuracy |
| ----- | ---------- | -------------  | -----------   |
|     |    |.     |.     |

In [16]:
# knn section

from sklearn.neighbors import KNeighborsClassifier
knn_clf = KNeighborsClassifier(n_neighbors=5)
start_time_knn = time.time()
knn_clf.fit(X_train, y_train)
end_time_knn = time.time()
y_pred_knn = knn_clf.predict(X_test)

time_trained_knn = end_time_knn - start_time_knn
print(f"Training Time: {time_trained_knn:.2f} seconds")

# model evaluation
train_accuracy_knn = accuracy_score(y_train, knn_clf.predict(X_train))
accuracy_knn = accuracy_score(y_test, y_pred_knn)
print(f"Training Accuracy: {train_accuracy_knn:.4f}")
print(f"KNN Accuracy: {accuracy_knn:.4f}")
print("\nClassification Report:")
print(classification_report(y_test, y_pred_knn))

Training Time: 0.01 seconds
Training Accuracy: 0.9200
KNN Accuracy: 0.8985

Classification Report:
              precision    recall  f1-score   support

           0       0.92      0.97      0.94      6797
           1       0.57      0.37      0.45       852

    accuracy                           0.90      7649
   macro avg       0.75      0.67      0.69      7649
weighted avg       0.88      0.90      0.89      7649



In [17]:
# SVM section

from sklearn.svm import SVC
svm_clf = SVC(kernel='rbf', random_state=42)
start_time_svm = time.time()
svm_clf.fit(X_train, y_train)
end_time_svm = time.time()
y_pred_svm = svm_clf.predict(X_test)

time_trained_svm = end_time_svm - start_time_svm
print(f"Training Time: {time_trained_svm:.2f} seconds")

# model evaluation
train_accurarcy_svm = accuracy_score(y_train, svm_clf.predict(X_train))
accuracy_svm = accuracy_score(y_test, y_pred_svm)
print(f"Training Accuracy: {train_accurarcy_svm:.4f}")
print(f"SVM Accuracy: {accuracy_svm:.4f}")
print("\nClassification Report:")
print(classification_report(y_test, y_pred_svm))

Training Time: 5.38 seconds
Training Accuracy: 0.9050
SVM Accuracy: 0.9051

Classification Report:
              precision    recall  f1-score   support

           0       0.91      0.98      0.95      6797
           1       0.69      0.27      0.39       852

    accuracy                           0.91      7649
   macro avg       0.80      0.63      0.67      7649
weighted avg       0.89      0.91      0.89      7649



In [19]:
perf_df = pd.DataFrame({
    'Model': ['Logistic Regression', 'KNN', 'SVM'],
    'Train Time': [time_trained_log_reg, time_trained_knn, time_trained_svm],
    'Train Accuracy': [train_accurarcy_log_reg, train_accuracy_knn, train_accurarcy_svm],
    'Test Accuracy': [accuracy_log_reg, accuracy_knn, accuracy_svm]
})

perf_df

,Model,Train Time,Train Accuracy,Test Accuracy
0,Logistic Regression,1.055034,0.905576,0.906785
1,KNN,0.006000,0.919990,0.898549
2,SVM,5.384274,0.904955,0.905086


In [20]:
# From the result df above, we can tell that KNN takes the shortest time for training, but its acc score is the worst. SVM took the most time but its score is worse than logistic regression.
# Logistic regression has a faster training time with the best test accurarcy, which could be the optimal solution for our problem.

### Problem 11: Improving the Model

Now that we have some basic models on the board, we want to try to improve these.  Below, we list a few things to explore in this pursuit.

- More feature engineering and exploration.  For example, should we keep the gender feature?  Why or why not?
- Hyperparameter tuning and grid search.  All of our models have additional hyperparameters to tune and explore.  For example the number of neighbors in KNN or the maximum depth of a Decision Tree.  
- Adjust your performance metric

In [21]:
# More feature engineering and exploration. For example, should we keep the gender feature? Why or why not?
# I think my answer would be no. Gender won't be a good feature that has a strong correlation. Since different people can have difference decisions on keeping the term deposit
# without a gender distinction. Both male and females have the potential to accept it or refuse it.
# In my experiments above, I already think of the features that would serve robust and strong params for prediction, so I don't want to modify this.

In [22]:
# Hyperparameter tuning and grid search.

In [23]:
# KNN tuning
from sklearn.model_selection import GridSearchCV
from sklearn.neighbors import KNeighborsClassifier

param_grid = {
    'n_neighbors': [3, 5, 7, 9],
    'weights': ['uniform', 'distance']
}

grid_search_knn = GridSearchCV(KNeighborsClassifier(), param_grid, cv=5, scoring='accuracy')
grid_search_knn.fit(X_train, y_train)

print("Best parameters:", grid_search_knn.best_params_)
print("Best cross-validation score:", grid_search_knn.best_score_)

# from the tuning result, I can see n_neighbors=9 with uniform weight would give the optimal result. The tuning actually increases our accurarcy from 0.8985 to 0.9019

Best parameters: {'n_neighbors': 9, 'weights': 'uniform'}
Best cross-validation score: 0.9019479826576815


In [ ]:
# SVM tuning
from sklearn.model_selection import GridSearchCV
from sklearn.svm import SVC

# Define parameter grid
param_grid = [
    {'kernel': ['linear'],},
    {'kernel': ['rbf'],},
#    {'kernel': ['poly'], 'degree': [2,3]},
    {'kernel': ['sigmoid']}
]

# Create the SVC model
svc = SVC()

# Grid search with 5-fold cross-validation
grid_search_svc = GridSearchCV(svc, param_grid, cv=5, scoring='accuracy', n_jobs=-1,verbose=2)
grid_search_svc.fit(X_train, y_train)

# Results
print("Best Parameters:", grid_search_svc.best_params_)
print(f"Best Cross-Validation Score: {grid_search_svc.best_score_:.4f}")

# not sure why this takes forever to train, so I can give it up.


Fitting 5 folds for each of 3 candidates, totalling 15 fits


KeyboardInterrupt: 

In [25]:
# Adjust your performance metric
# the classification report already gives matrix on different metrics, I don't think I can have other metrics that can best evaluate the model' performance other than that
# I will keep using the accuracy score since this is the most common and evident one.

##### Questions